
#Finetuning U2net

참고: https://github.com/danielgatis/rembg/issues/193#issuecomment-1055534289

In [ ]:
import os
os.chdir('/home/aidev/hy/ft_u2net/U-2-Net-master')

## u2net_train.py와 동일하게 진행

0. import

In [ ]:
import os
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim
import torchvision.transforms as standard_transforms

import numpy as np
import glob
import os
import gdown

from data_loader import Rescale
from data_loader import RescaleT
from data_loader import RandomCrop
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

from model import U2NET
from model import U2NETP


1. define loss function

In [ ]:
bce_loss = nn.BCELoss(size_average=True)

def muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v):

	loss0 = bce_loss(d0,labels_v)
	loss1 = bce_loss(d1,labels_v)
	loss2 = bce_loss(d2,labels_v)
	loss3 = bce_loss(d3,labels_v)
	loss4 = bce_loss(d4,labels_v)
	loss5 = bce_loss(d5,labels_v)
	loss6 = bce_loss(d6,labels_v)

	loss = loss0 + loss1 + loss2 + loss3 + loss4 + loss5 + loss6
	print("l0: %3f, l1: %3f, l2: %3f, l3: %3f, l4: %3f, l5: %3f, l6: %3f\n"%(loss0.data.item(),loss1.data.item(),loss2.data.item(),loss3.data.item(),loss4.data.item(),loss5.data.item(),loss6.data.item()))

	return loss0, loss

/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


(prepare training dataset(background-removed picture to mask)
배경=0, 물체=1)

2. set the directory of training dataset

In [ ]:
model_name = 'u2net' #'u2netp'

data_dir = os.path.join(os.getcwd(), 'finetune_data' + os.sep) #/content/drive/MyDrive/U-2-Net-master/finetune_data/
tra_image_dir = os.path.join("finetune_input"+ os.sep) #이부분 수정
tra_label_dir = os.path.join("finetune_groundtruth"+ os.sep) #이부분 수정
print('data_dir:',data_dir)

image_ext = '.png'
label_ext = '.png'

model_dir = os.path.join(os.getcwd(), 'saved_models', model_name + os.sep) #/content/drive/MyDrive/U-2-Net-master/saved_models/u2net/
print('model_dir',model_dir)

epoch_num = 100000
batch_size_train = 12
batch_size_val = 1
train_num = 0
val_num = 0

tra_img_name_list = glob.glob(data_dir + tra_image_dir + '*' + image_ext)
print(tra_img_name_list)
tra_lbl_name_list = glob.glob(data_dir + tra_label_dir + '*' + image_ext)
print(tra_lbl_name_list)

print("---")
print("train images: ", len(tra_img_name_list))
print("train labels: ", len(tra_lbl_name_list))
print("---")

train_num = len(tra_img_name_list)

salobj_dataset = SalObjDataset(
    img_name_list=tra_img_name_list,
    lbl_name_list=tra_lbl_name_list,
    transform=transforms.Compose([
        RescaleT(320),
        RandomCrop(288),
        ToTensorLab(flag=0)]))
salobj_dataloader = DataLoader(salobj_dataset, batch_size=batch_size_train, shuffle=True, num_workers=1)


data_dir: /content/drive/MyDrive/U-2-Net-master/finetune_data/
model_dir /content/drive/MyDrive/U-2-Net-master/saved_models/u2net/
['/content/drive/MyDrive/U-2-Net-master/finetune_data/finetune_input/강아지 (1).png']
['/content/drive/MyDrive/U-2-Net-master/finetune_data/finetune_groundtruth/강아지 (1)_clipdrop-background-removal.png']
---
train images:  1
train labels:  1
---


3. define model

In [ ]:
#bring state_dict model
if(model_name=='u2net'):
    net = U2NET(3, 1)
elif(model_name=='u2netp'):
    net = U2NETP(3,1)

#get u2net check point(pth file)

os.makedirs('./saved_models/u2net', exist_ok=True)

gdown.download('https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ',
    './saved_models/u2net/u2net.pth',
    quiet=False)

net.load_state_dict(torch.load('saved_models/u2net/u2net.pth'))

"""또는 하단 코드
checkpoint = torch.load(pathlib.Path(model_path))
net.load_state_dict(checkpoint)
net.load_state_dict(torch.load(model_path))
"""

if torch.cuda.is_available():
    net.cuda()

Downloading...
From: https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ
To: /content/drive/MyDrive/U-2-Net-master/saved_models/u2net/u2net.pth
 76%|███████▋  | 135M/176M [00:03<00:00, 42.6MB/s]

KeyboardInterrupt: ignored

4. define optimizer

In [ ]:
print("---define optimizer...")
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)


---define optimizer...


5. training process

In [ ]:
print("---start training...")
ite_num = 0
running_loss = 0.0
running_tar_loss = 0.0
ite_num4val = 0
save_frq = 2000 # save the model every 2000 iterations

for epoch in range(0, epoch_num):
    net.train()

    for i, data in enumerate(salobj_dataloader):
        ite_num = ite_num + 1
        ite_num4val = ite_num4val + 1

        inputs, labels = data['image'], data['label']

        inputs = inputs.type(torch.FloatTensor)
        labels = labels.type(torch.FloatTensor)

        # wrap them in Variable
        if torch.cuda.is_available():
            inputs_v, labels_v = Variable(inputs.cuda(), requires_grad=False), Variable(labels.cuda(),
                                                                                        requires_grad=False)
        else:
            inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)

        # y zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        d0, d1, d2, d3, d4, d5, d6 = net(inputs_v)
        loss2, loss = muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v)

        loss.backward()
        optimizer.step()

        # # print statistics
        running_loss += loss.data.item()
        running_tar_loss += loss2.data.item()

        # del temporary outputs and loss
        del d0, d1, d2, d3, d4, d5, d6, loss2, loss

        print("[epoch: %3d/%3d, batch: %5d/%5d, ite: %d] train loss: %3f, tar: %3f " % (
        epoch + 1, epoch_num, (i + 1) * batch_size_train, train_num, ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))

        if ite_num % save_frq == 0:

            torch.save(net.state_dict(), model_dir + model_name+"_bce_itr_%d_train_%3f_tar_%3f.pth" % (ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))
            running_loss = 0.0
            running_tar_loss = 0.0
            net.train()  # resume train
            ite_num4val = 0


---start training...


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:3737: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

[epoch: 15441/100000, batch:    12/    1, ite: 15441] train loss: 0.118676, tar: 0.000082 
l0: 0.000067, l1: 0.000066, l2: 0.001150, l3: 0.011318, l4: 0.019767, l5: 0.032365, l6: 0.045080

[epoch: 15442/100000, batch:    12/    1, ite: 15442] train loss: 0.118670, tar: 0.000082 
l0: 0.000071, l1: 0.000071, l2: 0.001156, l3: 0.011856, l4: 0.020443, l5: 0.032613, l6: 0.062764

[epoch: 15443/100000, batch:    12/    1, ite: 15443] train loss: 0.118677, tar: 0.000082 
l0: 0.000071, l1: 0.000072, l2: 0.001169, l3: 0.011532, l4: 0.019845, l5: 0.031504, l6: 0.046596

[epoch: 15444/100000, batch:    12/    1, ite: 15444] train loss: 0.118672, tar: 0.000082 
l0: 0.000071, l1: 0.000072, l2: 0.001451, l3: 0.011661, l4: 0.019785, l5: 0.033402, l6: 0.048626

[epoch: 15445/100000, batch:    12/    1, ite: 15445] train loss: 0.118669, tar: 0.000082 
l0: 0.000071, l1: 0.000071, l2: 0.001099, l3: 0.011423, l4: 0.019399, l5: 0.030083, l6: 0.046226

[epoch: 15446/1000

now to apply rembg, try another code (run_rembg_with_ft_u2net.ipynb)